In [ ]:
import weaviate
from weaviate.config import AdditionalConfig
from weaviate.classes.config import Configure

import json
import pandas as pd
import numpy as np
import os
import dotenv


from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_openai import AzureOpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai.chat_models import AzureChatOpenAI  

dotenv.load_dotenv("/home/isma/repos/book/data-pipelines-with-airflow-2nd-ed/chapter13_genai/.env")

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')
AZURE_OPENAI_RESOURCE_NAME = os.getenv('AZURE_OPENAI_RESOURCE_NAME')


client = weaviate.connect_to_local(
    port=8083,
    grpc_port=50051,
    headers = {
        "X-Azure-Api-Key": AZURE_OPENAI_API_KEY,
    },
)

print(client.is_ready())


In [ ]:
df = pd.read_parquet("compared.parquet")
df

In [ ]:
print(df)

In [ ]:
from weaviate.classes.query import Filter


In [ ]:
COLLECTION_NAME = "recipes"
collection = client.collections.get(name=COLLECTION_NAME)

for item in collection.iterator():
    print(item.uuid, item.properties)

In [ ]:
COLLECTION_NAME = "recipes"
collection = client.collections.get(name=COLLECTION_NAME)

for document_uuid in df.document_sha.unique():

    collection.data.delete_many(
        where=Filter.by_property("document_sha").equal(document_uuid)
    )

In [ ]:
documents_to_update = pd.DataFrame()
documents_to_create = pd.DataFrame()

for document in df.document_sha.unique():

    documents = df[df.document_sha == document]

    response = collection.query.fetch_objects(
    filters=Filter.by_property("document_sha").equal(document),
    )

    keys_in_db = [str(object.uuid) for object in response.objects]

    if len(keys_in_db) == 0:
        print("1")
        documents_to_create = append_rows_to_dataframe(documents_to_update, documents)        

    elif set(documents.chunk_sha) != set(keys_in_db):
        print("3")
        documents_to_update = append_rows_to_dataframe(documents_to_update, documents)

    break


In [ ]:
documents_to_update

In [ ]:
keys_in_db

In [ ]:
set(documents.chunk_sha) != set(keys_in_db)

In [ ]:
documents_to_update

In [ ]:
x = ['a', 'b']
y = ['b', 'a']

 # prefer this if elements are hashable


In [ ]:
client.collections.delete(name=COLLECTION_NAME)


In [ ]:
import pandas as pd
df = pd.read_parquet("compared.parquet")
df

In [ ]:
pd.concat([df, df])